In [1]:
# imports
import numpy as np
import tensorflow as tf

In [ ]:
import numpy as np

In [ ]:
# Batch Generation
def get_batch(vectorized_songs, seq_length, batch_size):
  
  # the length of the vectorized songs string
  n = vectorized_songs.shape[0] - 1

  # randomly choose the starting indices for the examples in the training batch
  idx = np.random.choice(n-seq_length, batch_size)

  input_batch = [vectorized_songs[i : i+seq_length] for i in idx]
  output_batch = [vectorized_songs[i+1 : i+seq_length+1] for i in idx]
  

  # x_batch, y_batch provide the true inputs and targets for network training
  x_batch = np.reshape(input_batch, [batch_size, seq_length])
  y_batch = np.reshape(output_batch, [batch_size, seq_length])
  return x_batch, y_batch

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adagrad, Adam
import numpy as np

def build_audio_generation_model(batch_size, vocab_size, embedding_dim):
    model = Sequential()

    # Embedding layer
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, batch_input_shape=[batch_size, None]))

    # LSTM layers
    model.add(LSTM(units=128, return_sequences=True,recurrent_initializer='glorot_uniform', recurrent_activation='sigmoid', stateful=True,))
    model.add(LSTM(units=256, return_sequences=True, recurrent_initializer='glorot_uniform', recurrent_activation='sigmoid', stateful=True,))
    
    # Dense layer for output
    model.add(Dense(units=1, activation='linear'))  # Assuming regression output, change units to 1
    model.compile(loss='mse', optimizer=Adam(5e-2))  # Use mean squared error for regression task

    return model

model = build_audio_generation_model(batch_size=1, vocab_size=64, embedding_dim=128)
model.summary()

In [ ]:
seq_length = 1000
batch_size = 1

x_batch, y_batch = get_batch(mfccs, seq_length, batch_size)
print(x_batch.shape, y_batch.shape)
model.fit(x_batch, y_batch, epochs=1000, batch_size=batch_size)

In [ ]:
def generate_mfcc():
    x_test, y_test = get_batch(mfccs, seq_length= 1000, batch_size=1)
    predictions = model.predict(x_test)
    return y_test

In [ ]:
prec = generate_mfcc()
prec.shape

In [ ]:
prec = generate_mfcc()
waveform = librosa.feature.inverse.mfcc_to_audio(prec)
import IPython.display as ipd
ipd.Audio(waveform, rate=sampling_rate)
